In [1]:
# libraries
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)
import seaborn as sns

import time
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action="ignore", module="sklearn", message="^n_iter")

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from catboost import CatBoostClassifier
from sklearn.utils import resample
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif, RFECV, SelectKBest, SelectFromModel
from sklearn.metrics import accuracy_score, classification_report, roc_curve, auc, confusion_matrix, log_loss, f1_score
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, KFold

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score, cross_validate, learning_curve

In [2]:
data = pd.read_csv("data.csv")
data.drop("Unnamed: 0", axis = 1, inplace = True)

In [3]:
cat_target = data["default"]
cat_features = data.drop("default", axis = 1)
X_trainC, X_testC, Y_trainC, Y_testC = train_test_split(cat_features, cat_target, test_size=0.3, random_state = 42)

In [7]:
# Upsampling for imbalance
def binary_upsample(data, label):
    value_count = data[label].value_counts()
    frqs = data[label].value_counts().values
    idc = data[label].value_counts().index
    minorfrq = value_count[frqs == min(frqs)].index[0] 
    n_sample = max(frqs) - min(frqs)
    minorindex = data[label][data[label] == minorfrq].index.values
    resampled = resample(minorindex, n_samples = n_sample)
    _ = pd.DataFrame(list(data.index) + list(resampled), columns =["Index"])
    balanced = pd.merge(_, data, how ="left", left_on= "Index", right_on= data.index)
    return balanced

In [ ]:
# Dummify
dummed_data = pd.get_dummies(data, drop_first = True)
updsampled = binary_upsample(dummed_data, "default").reindex()
updsampled.drop("Index", axis = 1, inplace = True)

In [ ]:
target = updsampled["default"]
features = updsampled.drop("default", axis = 1)
X_train, X_test, Y_train, Y_test = train_test_split(features, target, test_size=0.3, random_state = 42)

In [ ]:
Y_train.mean()
Y_test.mean()

### Ramdon Forest

In [64]:
# Setup 
KF = KFold(3, shuffle = True, random_state= 42)
def RF(X, Y, X1, Y1, params):
    start = time.time()
    RF = RandomForestClassifier(n_jobs = -1, oob_score = True,random_state = 42, class_weight= "balanced", 
                                n_estimators = params[0],
                                min_samples_split = params[1],
                                min_samples_leaf = params[2],
                                max_samples = params[3],
                                max_leaf_nodes = params[4],
                                max_depth = params[5]).fit(X,Y)
    stop = time.time()
    Time = stop - start
    TrainCV = cross_val_score(RF, X, Y, cv=KF, scoring = "f1", n_jobs = -1)
    train_F1Score = f1_score(Y, RF.predict(X))
    train_ac = RF.score(X, Y)
    Train_CM = confusion_matrix(Y, RF.predict(X))
    
    TestCV = cross_val_score(RF, X1, Y1, cv=KF, scoring = "f1", n_jobs = -1)
    test_F1Score = f1_score(Y1, RF.predict(X1))
    tes_ac = RF.score(X1, Y1) 
    Test_CM = confusion_matrix(Y1, RF.predict(X1))
    
    return TrainCV, train_F1Score , train_ac, Train_CM, TestCV, test_F1Score, tes_ac, Test_CM, Time

In [80]:
# Ramdonized Search
pipeline = Pipeline([("EF",RandomForestClassifier(n_jobs = -1, oob_score = True, 
                                                  random_state = 42, class_weight= "balanced"))])
param_grid = {
                  "EF__max_depth": [10, 50, 100, 150, 200, 250], 
                  "EF__max_samples": [0.2, 0.4, 0.6, 0.8],
                  "EF__min_samples_leaf": [10, 50, 100], 
                  "EF__min_samples_split": [10, 50, 100],
                  "EF__n_estimators" : [50, 100, 200, 400, 1000],
                  "EF__max_leaf_nodes" : [10, 20, 50, 100]
    
                  }                                               
# pipeline.get_params().keys()
search = RandomizedSearchCV(
    estimator = pipeline,
    param_distributions = param_grid,
    n_iter = 100, 
    return_train_score = True,
    n_jobs= -1,
    scoring="f1",
    cv = 5,
    verbose=3)
  
search = search.fit(X_train, Y_train) 
print(search.best_params_, search.best_score_)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
{'EF__n_estimators': 100, 'EF__min_samples_split': 50, 'EF__min_samples_leaf': 100, 'EF__max_samples': 0.2, 'EF__max_leaf_nodes': 100, 'EF__max_depth': 200} 0.6478119837534354


In [79]:
RF(X_train, Y_train, X_test, Y_test, [100, 2, 2, 0.2, 5, 10])

(array([0.63220935, 0.63254681, 0.62926675]),
 0.6323907117285774,
 0.6414857428393883,
 array([[51998, 25856],
        [30107, 48136]]),
 array([0.62176795, 0.63018992, 0.63271393]),
 0.6321031678937579,
 0.642221856828951,
 array([[22402, 11242],
        [12693, 20562]]),
 4.838979721069336)

In [51]:
# Ramdonized Search
pipeline = Pipeline([('selector', SelectKBest(chi2)),
                     ("EF",RandomForestClassifier(n_jobs = -1, oob_score = True, random_state = 42,  class_weight= "balanced",))])
param_grid = {
                  "EF__max_depth": [3, 5, 10, 15, 20], 
                  "EF__max_samples": [0.1, 0.2, 0.3, 0.4],
                  "EF__min_samples_leaf": [2, 4 ,8], 
                  "EF__min_samples_split": [2, 4 ,8],
                  "EF__n_estimators" : [20, 40, 80, 100],
                  "EF__max_features" : [2 , 4, 6, 10, 15, 20, 25],
                  "EF__max_leaf_nodes" : [5, 10, 15, 20],
                  "selector__k": range(50, 150, 10)
                  }                                               
# pipeline.get_params().keys()
search = RandomizedSearchCV(
    estimator = pipeline,
    param_distributions = param_grid,
    n_iter = 50, 
    return_train_score = True,
    n_jobs= -1,
    scoring="f1",
    cv = 5,
    verbose=3)

search = search.fit(X_train, Y_train)
print(search.best_params_, search.best_score_)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
{'selector__k': 140, 'EF__n_estimators': 20, 'EF__min_samples_split': 8, 'EF__min_samples_leaf': 8, 'EF__max_samples': 0.1, 'EF__max_leaf_nodes': 5, 'EF__max_features': 6, 'EF__max_depth': 20} 0.6486812440819931


In [53]:
chi_train = SelectKBest(chi2, k = 140).fit_transform(X_train, Y_train)
chi_test = SelectKBest(chi2, k = 140).fit_transform(X_test, Y_test)
RF(chi_train, Y_train, chi_test, Y_test, [20, 8, 8, 0.1, 5, 6, 20])

(array([0.64942974, 0.6597975 , 0.64534117]),
 0.6470162838831949,
 0.6372768214635771,
 array([[47585, 30269],
        [26351, 51892]]),
 array([0.61600262, 0.63768784, 0.6319706 ]),
 0.6513624642832253,
 0.6261080135726992,
 array([[18520, 15124],
        [ 9889, 23366]]),
 1.0207369327545166)

In [2]:
47585/(47585+30269)

0.6112081588614586

In [54]:
# Ramdonized Search
pipeline = Pipeline([('selector', SelectKBest(f_classif)),
                     ("EF",RandomForestClassifier(n_jobs = -1, oob_score = True, random_state = 42,  class_weight= "balanced"))])
param_grid = {
                  "EF__max_depth": [3, 5, 10, 15, 20], 
                  "EF__max_samples": [0.1, 0.2, 0.3, 0.4],
                  "EF__min_samples_leaf": [2, 4 ,8], 
                  "EF__min_samples_split": [2, 4 ,8],
                  "EF__n_estimators" : [20, 40, 80, 100, 120],
                  "EF__max_features" : [2 , 4, 6, 10, 15, 20, 25],
                  "EF__max_leaf_nodes" : [5, 10, 15, 20],
                  "selector__k": range(50, 150, 10)
                  }                                               
# pipeline.get_params().keys()
search = RandomizedSearchCV(
    estimator = pipeline,
    param_distributions = param_grid,
    n_iter = 50, 
    return_train_score = True,
    n_jobs= -1,
    scoring="f1",
    cv = 5,
    verbose=3)

search = search.fit(X_train, Y_train)
print(search.best_params_, search.best_score_)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
{'selector__k': 140, 'EF__n_estimators': 40, 'EF__min_samples_split': 4, 'EF__min_samples_leaf': 4, 'EF__max_samples': 0.2, 'EF__max_leaf_nodes': 15, 'EF__max_features': 6, 'EF__max_depth': 10} 0.640545694817667


In [56]:
f_train = SelectKBest(f_classif, k = 140).fit_transform(X_train, Y_train)
f_test = SelectKBest(f_classif, k = 140).fit_transform(X_test, Y_test)
RF(f_train, Y_train, f_test, Y_test, [40, 4, 4, 0.2, 15, 6, 10])

(array([0.6419609 , 0.64820115, 0.63761377]),
 0.6455754538786584,
 0.6501982741500477,
 array([[51765, 26089],
        [28514, 49729]]),
 array([0.62933009, 0.64583236, 0.6340309 ]),
 0.6003692348267564,
 0.6376029537063335,
 array([[24444,  9200],
        [15044, 18211]]),
 2.0121419429779053)

In [57]:
# Ramdonized Search
pipeline = Pipeline([('selector', SelectKBest(mutual_info_classif)),
                     ("EF",RandomForestClassifier(n_jobs = -1, oob_score = True, random_state = 42,  class_weight= "balanced"))])
param_grid = {
                  "EF__max_depth": [3, 5, 10, 15, 20], 
                  "EF__max_samples": [0.1, 0.2, 0.3, 0.4],
                  "EF__min_samples_leaf": [2, 4 ,8], 
                  "EF__min_samples_split": [2, 4 ,8],
                  "EF__n_estimators" : [20, 40, 80, 100, 120],
                  "EF__max_features" : [2 , 4, 6, 10, 15, 20, 25],
                  "EF__max_leaf_nodes" : [5, 10, 15, 20],
                  "selector__k": range(50, 150, 10)
                  }                                               
# pipeline.get_params().keys()
search = RandomizedSearchCV(
    estimator = pipeline,
    param_distributions = param_grid,
    n_iter = 10, 
    return_train_score = True,
    n_jobs= -1,
    scoring="f1",
    cv = 5,
    verbose=3)

search = search.fit(X_train, Y_train)
print(search.best_params_, search.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
{'selector__k': 80, 'EF__n_estimators': 100, 'EF__min_samples_split': 8, 'EF__min_samples_leaf': 8, 'EF__max_samples': 0.2, 'EF__max_leaf_nodes': 10, 'EF__max_features': 2, 'EF__max_depth': 15} 0.6411369794751092


In [ ]:
MI_train = SelectKBest(mutual_info_classif, k = 80).fit_transform(X_train, Y_train),
MI_test = SelectKBest(mutual_info_classif, k = 80).fit_transform(X_test, Y_test)
RF(MI_train, Y_train, MI_test, Y_test, [100, 8, 8, 0.2, 10, 2, 15])

In [83]:
# RFE - Orginal model
estimator = RandomForestClassifier(n_jobs = -1, oob_score = True,random_state = 42, class_weight= "balanced", 
                                n_estimators = 100,
                                min_samples_split = 2,
                                min_samples_leaf = 2,
                                max_samples = 0.1,
                                max_leaf_nodes = 10,
                                max_depth = 4
                                )
selector = RFECV(estimator, step = 1, cv = KF, n_jobs = -1, scoring = "f1").fit(X_train, Y_train)
print(selector.n_features_, selector.ranking_)

86 [ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  2  1 17  1 10  1
  1  1  1 46  1  1  1  1  1  1  1  1  1 28  7  1  1  1  1  1 31  1  1  1
  1  1  1  1 44 38 36 40 45  1 14  1 20  1 25  1 33 47 49 51  1 54  1 23
 61  1  1  1  9 71 70  3  1  1 26 11 24  1 12 29  5  1 32  1  1  1  1  1
 16  1  1  1  1  1  1  1  1  1 42 18  1  1  1  1  8 39 34 27 37  6 35 30
  1 22  1 50 41  4 15  1  1  1  1  1  1  1 13  1 19 21 43 48 52 53 55 56
 57 58 59 60 62 63 64 65 66 67 68 69]


In [84]:
RFE = pd.DataFrame(selector.support_, index = X_train.columns).rename(columns = {0:"selected"})  
RFE_train, RFE_test = X_train[RFE[RFE.selected == True].index], X_test[RFE[RFE.selected == True].index]
RF(RFE_train, Y_train, RFE_test, Y_test, [100, 2, 2, 0.1, 10, 4, 15])

(array([0.63617307, 0.63652923, 0.63464654]),
 0.6357536528673117,
 0.6456241952119516,
 array([[52505, 25349],
        [29968, 48275]]),
 array([0.62306433, 0.63440045, 0.63604143]),
 0.6350589757013951,
 0.6457346148671879,
 array([[22578, 11066],
        [12634, 20621]]),
 3.233290910720825)

In [81]:
# Select from moel - logsitic
Model_sel = SelectFromModel(estimator = estimator).fit(X_train, Y_train)
Model_selected = pd.DataFrame(Model_sel.get_support(), index = X_train.columns).rename(columns = {0: "selected"})
ms_train, ms_test = X_train[Model_selected[Model_selected.selected == True].index], X_test[Model_selected[Model_selected.selected == True].index]

In [82]:
RF(ms_train, Y_train, ms_test, Y_test, params = [100, 2, 2, 0.1, 10, 4, 15])

(array([0.62339345, 0.62502263, 0.62226167]),
 0.6264262939418854,
 0.6419726195891017,
 array([[53353, 24501],
        [31386, 46857]]),
 array([0.61037394, 0.62024457, 0.62592212]),
 0.6262951444779572,
 0.6425507107729562,
 array([[22948, 10696],
        [13217, 20038]]),
 1.9438858032226562)

### CatBoost

In [103]:
CATboost = CatBoostClassifier(random_seed = 42, boosting_type = "Plain", eval_metric = "F1", od_type = "IncToDec", 
                              bootstrap_type = "Bernoulli")
grid = {"iterations" :[100, 500, 1000, 2000],
        "learning_rate": [0.0003, 0.003, 0.03, 0.3],
        "l2_leaf_reg": [3, 9, 18, 25, 35],
        "depth" : [10, 12 , 14 ,16],
        "rsm": [0.3, 0.5, 0.7],
        "min_data_in_leaf": [2, 4, 8, 10], 
       }

randomized_search_result = CATboost.randomized_search(grid,
                                                       X= X_train,
                                                       y= Y_train,
                                                       cv = KF, 
                                                       n_iter = 10,
                                                       shuffle = True,
                                                       search_by_train_test_split = True,
                                                       plot=True)

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6462352	test: 0.6378378	best: 0.6378378 (0)	total: 310ms	remaining: 2m 34s
1:	learn: 0.6752203	test: 0.6542461	best: 0.6542461 (1)	total: 638ms	remaining: 2m 38s
2:	learn: 0.6814327	test: 0.6588823	best: 0.6588823 (2)	total: 961ms	remaining: 2m 39s
3:	learn: 0.6895746	test: 0.6641482	best: 0.6641482 (3)	total: 1.32s	remaining: 2m 43s
4:	learn: 0.7031194	test: 0.6730592	best: 0.6730592 (4)	total: 1.65s	remaining: 2m 43s
5:	learn: 0.7163041	test: 0.6815082	best: 0.6815082 (5)	total: 1.98s	remaining: 2m 42s
6:	learn: 0.7248423	test: 0.6862996	best: 0.6862996 (6)	total: 2.3s	remaining: 2m 42s
7:	learn: 0.7277466	test: 0.6875555	best: 0.6875555 (7)	total: 2.61s	remaining: 2m 40s
8:	learn: 0.7323937	test: 0.6910681	best: 0.6910681 (8)	total: 2.94s	remaining: 2m 40s
9:	learn: 0.7400513	test: 0.6950833	best: 0.6950833 (9)	total: 3.28s	remaining: 2m 40s
10:	learn: 0.7471110	test: 0.7000989	best: 0.7000989 (10)	total: 3.61s	remaining: 2m 40s
11:	learn: 0.7504588	test: 0.7010445	best:

93:	learn: 0.9575883	test: 0.8206451	best: 0.8206451 (93)	total: 28.8s	remaining: 2m 4s
94:	learn: 0.9588579	test: 0.8212489	best: 0.8212489 (94)	total: 29.1s	remaining: 2m 4s
95:	learn: 0.9596698	test: 0.8219785	best: 0.8219785 (95)	total: 29.4s	remaining: 2m 3s
96:	learn: 0.9603186	test: 0.8235176	best: 0.8235176 (96)	total: 29.7s	remaining: 2m 3s
97:	learn: 0.9611174	test: 0.8237345	best: 0.8237345 (97)	total: 30s	remaining: 2m 3s
98:	learn: 0.9622740	test: 0.8242083	best: 0.8242083 (98)	total: 30.3s	remaining: 2m 2s
99:	learn: 0.9626182	test: 0.8245002	best: 0.8245002 (99)	total: 30.6s	remaining: 2m 2s
100:	learn: 0.9638572	test: 0.8257939	best: 0.8257939 (100)	total: 30.9s	remaining: 2m 2s
101:	learn: 0.9654796	test: 0.8271766	best: 0.8271766 (101)	total: 31.3s	remaining: 2m 1s
102:	learn: 0.9667308	test: 0.8277298	best: 0.8277298 (102)	total: 31.6s	remaining: 2m 1s
103:	learn: 0.9682992	test: 0.8284409	best: 0.8284409 (103)	total: 31.9s	remaining: 2m 1s
104:	learn: 0.9696465	test

184:	learn: 0.9974997	test: 0.8693721	best: 0.8693721 (184)	total: 56.9s	remaining: 1m 36s
185:	learn: 0.9976354	test: 0.8702321	best: 0.8702321 (185)	total: 57.2s	remaining: 1m 36s
186:	learn: 0.9976513	test: 0.8702557	best: 0.8702557 (186)	total: 57.5s	remaining: 1m 36s
187:	learn: 0.9977072	test: 0.8712432	best: 0.8712432 (187)	total: 57.8s	remaining: 1m 35s
188:	learn: 0.9977391	test: 0.8713930	best: 0.8713930 (188)	total: 58.1s	remaining: 1m 35s
189:	learn: 0.9978509	test: 0.8717345	best: 0.8717345 (189)	total: 58.4s	remaining: 1m 35s
190:	learn: 0.9979226	test: 0.8716587	best: 0.8717345 (189)	total: 58.7s	remaining: 1m 35s
191:	learn: 0.9979146	test: 0.8712119	best: 0.8717345 (189)	total: 59s	remaining: 1m 34s
192:	learn: 0.9978986	test: 0.8716759	best: 0.8717345 (189)	total: 59.3s	remaining: 1m 34s
193:	learn: 0.9979546	test: 0.8713431	best: 0.8717345 (189)	total: 59.6s	remaining: 1m 34s
194:	learn: 0.9980185	test: 0.8718725	best: 0.8718725 (194)	total: 59.9s	remaining: 1m 33s
1

...

450:	learn: 1.0000000	test: 0.8948082	best: 0.8948082 (450)	total: 8m 35s	remaining: 29m 31s
451:	learn: 1.0000000	test: 0.8944198	best: 0.8948082 (450)	total: 8m 36s	remaining: 29m 30s
452:	learn: 1.0000000	test: 0.8945226	best: 0.8948082 (450)	total: 8m 37s	remaining: 29m 28s
453:	learn: 1.0000000	test: 0.8945818	best: 0.8948082 (450)	total: 8m 39s	remaining: 29m 27s
454:	learn: 1.0000000	test: 0.8947508	best: 0.8948082 (450)	total: 8m 40s	remaining: 29m 26s
455:	learn: 1.0000000	test: 0.8948692	best: 0.8948692 (455)	total: 8m 41s	remaining: 29m 25s
456:	learn: 1.0000000	test: 0.8947351	best: 0.8948692 (455)	total: 8m 42s	remaining: 29m 24s
457:	learn: 1.0000000	test: 0.8948291	best: 0.8948692 (455)	total: 8m 43s	remaining: 29m 23s
458:	learn: 1.0000000	test: 0.8951146	best: 0.8951146 (458)	total: 8m 44s	remaining: 29m 22s
459:	learn: 1.0000000	test: 0.8952312	best: 0.8952312 (459)	total: 8m 46s	remaining: 29m 21s
460:	learn: 1.0000000	test: 0.8952173	best: 0.8952312 (459)	total: 8m 

539:	learn: 1.0000000	test: 0.8972869	best: 0.8973284 (523)	total: 10m 22s	remaining: 28m 2s
540:	learn: 1.0000000	test: 0.8973005	best: 0.8973284 (523)	total: 10m 23s	remaining: 28m 2s
541:	learn: 1.0000000	test: 0.8971161	best: 0.8973284 (523)	total: 10m 25s	remaining: 28m 1s
542:	learn: 1.0000000	test: 0.8974207	best: 0.8974207 (542)	total: 10m 26s	remaining: 28m
543:	learn: 1.0000000	test: 0.8972252	best: 0.8974207 (542)	total: 10m 27s	remaining: 27m 59s
544:	learn: 1.0000000	test: 0.8972980	best: 0.8974207 (542)	total: 10m 28s	remaining: 27m 58s
545:	learn: 1.0000000	test: 0.8972481	best: 0.8974207 (542)	total: 10m 29s	remaining: 27m 57s
546:	learn: 1.0000000	test: 0.8973208	best: 0.8974207 (542)	total: 10m 31s	remaining: 27m 56s
547:	learn: 1.0000000	test: 0.8973369	best: 0.8974207 (542)	total: 10m 32s	remaining: 27m 55s
548:	learn: 1.0000000	test: 0.8970729	best: 0.8974207 (542)	total: 10m 33s	remaining: 27m 54s
549:	learn: 1.0000000	test: 0.8974232	best: 0.8974232 (549)	total: 

627:	learn: 1.0000000	test: 0.8994270	best: 0.8997991 (621)	total: 12m 5s	remaining: 26m 24s
628:	learn: 1.0000000	test: 0.8994929	best: 0.8997991 (621)	total: 12m 6s	remaining: 26m 23s
629:	learn: 1.0000000	test: 0.8993182	best: 0.8997991 (621)	total: 12m 7s	remaining: 26m 22s
630:	learn: 1.0000000	test: 0.8993611	best: 0.8997991 (621)	total: 12m 8s	remaining: 26m 21s
631:	learn: 1.0000000	test: 0.8992687	best: 0.8997991 (621)	total: 12m 10s	remaining: 26m 20s
632:	learn: 1.0000000	test: 0.8993412	best: 0.8997991 (621)	total: 12m 11s	remaining: 26m 19s
633:	learn: 1.0000000	test: 0.8991303	best: 0.8997991 (621)	total: 12m 12s	remaining: 26m 18s
634:	learn: 1.0000000	test: 0.8990511	best: 0.8997991 (621)	total: 12m 13s	remaining: 26m 17s
635:	learn: 1.0000000	test: 0.8991436	best: 0.8997991 (621)	total: 12m 14s	remaining: 26m 16s
636:	learn: 1.0000000	test: 0.8992294	best: 0.8997991 (621)	total: 12m 16s	remaining: 26m 14s
637:	learn: 1.0000000	test: 0.8991865	best: 0.8997991 (621)	tota

715:	learn: 1.0000000	test: 0.9006052	best: 0.9008221 (695)	total: 13m 46s	remaining: 24m 41s
716:	learn: 1.0000000	test: 0.9007498	best: 0.9008221 (695)	total: 13m 47s	remaining: 24m 40s
717:	learn: 1.0000000	test: 0.9007071	best: 0.9008221 (695)	total: 13m 48s	remaining: 24m 39s
718:	learn: 1.0000000	test: 0.9007729	best: 0.9008221 (695)	total: 13m 49s	remaining: 24m 38s
719:	learn: 1.0000000	test: 0.9009211	best: 0.9009211 (719)	total: 13m 50s	remaining: 24m 37s
720:	learn: 1.0000000	test: 0.9010230	best: 0.9010230 (720)	total: 13m 52s	remaining: 24m 35s
721:	learn: 1.0000000	test: 0.9010888	best: 0.9010888 (721)	total: 13m 53s	remaining: 24m 34s
722:	learn: 1.0000000	test: 0.9008061	best: 0.9010888 (721)	total: 13m 54s	remaining: 24m 33s
723:	learn: 1.0000000	test: 0.9008061	best: 0.9010888 (721)	total: 13m 54s	remaining: 24m 31s
724:	learn: 1.0000000	test: 0.9007337	best: 0.9010888 (721)	total: 13m 56s	remaining: 24m 30s
725:	learn: 1.0000000	test: 0.9008914	best: 0.9010888 (721)	

KeyboardInterrupt: 

In [5]:
categorical_features_indices = np.where(X_trainC.dtypes == "object")[0]
CATboost = CatBoostClassifier(eval_metric = "F1", random_seed = 42, verbose = False, auto_class_weights = "Balanced",
                              iterations = 200, learning_rate = 0.001, depth = 5, l2_leaf_reg = 5)
                             
CATboost.fit(X_trainC, Y_trainC, cat_features = categorical_features_indices, eval_set=(X_testC, Y_testC), plot=True)

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [7]:
print(CATboost.score(X_trainC, Y_trainC), CATboost.score(X_testC, Y_testC))
# X_trainC, X_testC, Y_trainC, Y_testC

0.9846537250786989 0.9841177797107052


In [8]:
confusion_matrix(Y_trainC, CATboost.predict(X_trainC))

array([[ 27485,    425],
       [  1798, 115148]])

In [10]:
27485/(27485+ 425)

0.9847724829810104